# AstrbotContainers 用法示例

这个笔记本演示了如何使用 AstrbotContainers 类来管理依赖注入的 provider。

In [1]:
# 导入必要的模块
import importlib
import sys

# Force complete reload
modules_to_remove = [k for k in sys.modules.keys() if k.startswith('astrbot_injector')]
for mod in modules_to_remove:
    del sys.modules[mod]

import astrbot_injector.containers
importlib.reload(astrbot_injector.containers)
from astrbot_injector.containers import AstrbotContainers
from dishka import Provider, provide, Scope, from_context

In [2]:
# 定义一个简单的 Provider
class ExampleProvider(Provider):
    @provide(scope=Scope.APP)
    def get_message(self) -> str:
        return "Hello from Astrbot!"

    @provide(scope=Scope.APP)
    def get_number(self) -> int:
        return 42

# 定义配置类型
class AppConfig:
    def __init__(self, message: str, version: str):
        self.message = message
        self.version = version

# 定义使用 from_context 的 Provider
class ContextProvider(Provider):
    scope = Scope.APP  # 改为 APP scope
    
    # 从上下文获取配置
    config = from_context(provides=AppConfig, scope=Scope.APP)
    
    @provide
    def get_config_message(self, config: AppConfig) -> str:
        return config.message

In [3]:
# 创建 AstrbotContainers 实例
containers = AstrbotContainers()

# 设置 provider: 使用 << 运算符，路径以 .p 结尾
result = containers << ("canary_root/message.p", ExampleProvider())
print(f"Result of << operation: {result}")
print(f"Is result the same as containers? {result is containers}")
print("Provider 'message.p' set in 'canary_root'.")

Result of << operation: <astrbot_injector.containers.AstrbotContainers object at 0x0000020B854AABA0>
Is result the same as containers? True
Provider 'message.p' set in 'canary_root'.


In [4]:
# 获取 provider: 使用 / 运算符，路径为 "container/provider.p"
print(f"Providers: {AstrbotContainers.providers}")
provider = AstrbotContainers.getProvider("canary_root", "message")
print(f"Direct getProvider: {provider}")
provider2 = (containers.sync_container / "canary_root/message.p").provider
print(f"Retrieved provider via /: {provider2}")

Providers: <astrbot_injector.containers.ContainerNode object at 0x0000020B854AAA50>
Direct getProvider: <__main__.ExampleProvider object at 0x0000020B854AAE40>
Retrieved provider via /: None


In [5]:
# 获取 provider: 使用 / 运算符，路径为 "container/provider.p"
print(f"Providers: {AstrbotContainers.providers}")
provider = AstrbotContainers.getProvider("canary_root", "message")
print(f"Direct getProvider: {provider}")
getter = containers.sync_container / "canary_root/message.p"
print(f"Getter path: {getter.path}")
try:
    provider2 = getter.provider
    print(f"Retrieved provider via /: {provider2}")
except Exception as e:
    print(f"Error: {e}")

Providers: <astrbot_injector.containers.ContainerNode object at 0x0000020B854AAA50>
Direct getProvider: <__main__.ExampleProvider object at 0x0000020B854AAE40>
Getter path: ['canary_root/message.p']
Retrieved provider via /: None


In [6]:
# 获取容器并使用依赖

# 设置根容器的 provider
_ = containers << ("canary_root/message.p", ExampleProvider())

# 设置子容器的 provider
_ = containers << ("canary_root/sub_container/data.p", ExampleProvider())

# 测试嵌套子节点访问
node = containers.sync_container / "canary_root"
print(f"Root node path: {node.path}")

sub_node = node / "sub_container"
print(f"Sub-node path: {sub_node.path}")

# 获取子容器的容器
sub_container = sub_node.container
print("Successfully accessed sub-container!")

# 使用子容器中的依赖
sub_message = sub_container.get(str)
sub_number = sub_container.get(int)
print(f"Message from sub-container: {sub_message}")
print(f"Number from sub-container: {sub_number}")

# 获取根容器
root_container = node.container
root_message = root_container.get(str)
root_number = root_container.get(int)
print(f"Message from root container: {root_message}")
print(f"Number from root container: {root_number}")

print("✓ Nested sub-nodes work perfectly!")

Root node path: ['canary_root']
Sub-node path: ['canary_root', 'sub_container']
Successfully accessed sub-container!
Message from sub-container: Hello from Astrbot!
Number from sub-container: 42
Message from root container: Hello from Astrbot!
Number from root container: 42
✓ Nested sub-nodes work perfectly!


In [7]:
# 移除 provider: 使用 >> 运算符
containers >> "canary_root/message.p"

In [8]:
# from_context 功能演示
print("=== from_context 功能演示 ===")

# 设置包含 from_context 的 provider
_ = containers << ("canary_root/context.p", ContextProvider())

# 创建带上下文的容器获取器
config_instance = AppConfig(message="Hello from context!", version="1.0")
context_getter = containers.sync_container_with_context({AppConfig: config_instance})

# 获取带上下文的容器
context_container = (context_getter / "canary_root").container

# 验证上下文数据已正确注入
injected_config = context_container.get(AppConfig)
print(f"注入的配置对象: {injected_config}")
print(f"配置消息: {injected_config.message}")
print(f"配置版本: {injected_config.version}")

# 验证这是同一个实例
print(f"这是同一个配置实例: {injected_config is config_instance}")

print("✓ from_context 功能集成成功!")

=== from_context 功能演示 ===
注入的配置对象: <__main__.AppConfig object at 0x0000020B854AA7B0>
配置消息: Hello from context!
配置版本: 1.0
这是同一个配置实例: True
✓ from_context 功能集成成功!


## 组件隔离演示

dishka 的组件功能允许在同一个容器中隔离不同的提供者组。

In [9]:
# 组件隔离演示
from typing import Annotated
from dishka import FromComponent

# 定义不同组件的 Provider
class UserProvider(Provider):
    component = "user"  # 指定组件名称
    
    @provide(scope=Scope.APP)
    def get_user_db_connection(self) -> str:
        return "User Database Connection"

class CommentProvider(Provider):
    component = "comment"  # 指定组件名称
    
    @provide(scope=Scope.APP)
    def get_comment_db_connection(self) -> str:
        return "Comment Database Connection"

# 默认组件的 Provider
class MainProvider(Provider):
    # 默认组件 (component = "")
    
    @provide(scope=Scope.APP)
    def get_user_data(self, conn: Annotated[str, FromComponent("user")]) -> str:
        return f"User data from: {conn}"
    
    @provide(scope=Scope.APP)
    def get_comment_data(self, conn: Annotated[str, FromComponent("comment")]) -> str:
        return f"Comment data from: {conn}"
    
    @provide(scope=Scope.APP)
    def get_isolated_data(self) -> str:
        return "Isolated data (default component only)"

# 设置提供者
containers = AstrbotContainers()
_ = containers << ("canary_root/user_db.p", UserProvider())
_ = containers << ("canary_root/comment_db.p", CommentProvider())
_ = containers << ("canary_root/main.p", MainProvider())

# 获取容器
container = containers.getContainer("canary_root")

print("=== 组件隔离演示 ===")

# 测试跨组件访问
user_data = container.get(str, component="user")  # UserProvider 的连接
print(f"用户数据库连接: {user_data}")

comment_data = container.get(str, component="comment")  # CommentProvider 的连接
print(f"评论数据库连接: {comment_data}")

# 测试 FromComponent 跨组件依赖注入
user_result = container.get(str)  # MainProvider.get_user_data
print(f"跨组件用户数据: {user_result}")

comment_result = container.get(str)  # MainProvider.get_comment_data  
print(f"跨组件评论数据: {comment_result}")

# 测试组件隔离 - 默认组件无法访问其他组件
try:
    isolated_data = container.get(str)  # MainProvider.get_isolated_data
    print(f"隔离数据: {isolated_data}")
except Exception as e:
    print(f"预期错误 (组件隔离生效): {e}")

print("✓ 组件隔离功能演示完成!")

DEBUG get_all_providers: self=2248498860816, self.providers={'message': None, 'context': <__main__.ContextProvider object at 0x0000020B85144EC0>, 'user_db': <__main__.UserProvider object at 0x0000020B854AAF90>, 'comment_db': <__main__.CommentProvider object at 0x0000020B854AB0E0>, 'main': <__main__.MainProvider object at 0x0000020B854AB230>}, all_providers={'message': None, 'context': <__main__.ContextProvider object at 0x0000020B85144EC0>, 'user_db': <__main__.UserProvider object at 0x0000020B854AAF90>, 'comment_db': <__main__.CommentProvider object at 0x0000020B854AB0E0>, 'main': <__main__.MainProvider object at 0x0000020B854AB230>}
DEBUG child sub_container: 2248504238288, providers={'data': <__main__.ExampleProvider object at 0x0000020B854BDF90>}
DEBUG get_all_providers: self=2248504238288, self.providers={'data': <__main__.ExampleProvider object at 0x0000020B854BDF90>}, all_providers={'data': <__main__.ExampleProvider object at 0x0000020B854BDF90>}
DEBUG final all_providers={'data

## 嵌套依赖和容器测试

测试 Provider 之间的依赖关系和 Container 的嵌套访问。

In [10]:
# 嵌套依赖和容器测试
from typing import Annotated

# 定义基础服务
class DatabaseService:
    def __init__(self, connection_string: str):
        self.connection_string = connection_string
    
    def connect(self) -> str:
        return f"Connected to {self.connection_string}"

class CacheService:
    def __init__(self, cache_size: int):
        self.cache_size = cache_size
    
    def get_cache_info(self) -> str:
        return f"Cache size: {self.cache_size}MB"

# 定义依赖于基础服务的高层服务
class UserRepository:
    def __init__(self, db: DatabaseService, cache: CacheService):
        self.db = db
        self.cache = cache
    
    def get_user(self, user_id: int) -> str:
        db_result = self.db.connect()
        cache_info = self.cache.get_cache_info()
        return f"User {user_id} from {db_result}, {cache_info}"

class UserService:
    def __init__(self, repo: UserRepository):
        self.repo = repo
    
    def get_user_profile(self, user_id: int) -> str:
        user_data = self.repo.get_user(user_id)
        return f"Profile: {user_data}"

# 定义 Provider
class BaseServicesProvider(Provider):
    component = "base"
    
    @provide(scope=Scope.APP)
    def database_service(self) -> DatabaseService:
        return DatabaseService("postgresql://localhost:5432/mydb")
    
    @provide(scope=Scope.APP)
    def cache_service(self) -> CacheService:
        return CacheService(512)

class RepositoryProvider(Provider):
    component = "repo"
    
    @provide(scope=Scope.APP)
    def user_repository(self, db: Annotated[DatabaseService, FromComponent("base")], 
                       cache: Annotated[CacheService, FromComponent("base")]) -> UserRepository:
        return UserRepository(db, cache)

class ServiceProvider(Provider):
    component = "service"
    
    @provide(scope=Scope.APP)
    def user_service(self, repo: Annotated[UserRepository, FromComponent("repo")]) -> UserService:
        return UserService(repo)

print("=== 嵌套依赖和容器测试 ===")

# 设置嵌套的 providers
containers = AstrbotContainers()
_ =containers << ("canary_root/base_services.p", BaseServicesProvider())
_ =containers << ("canary_root/repositories.p", RepositoryProvider())
_ =containers << ("canary_root/services.p", ServiceProvider())

# 获取根容器
root_container = containers.getContainer("canary_root")

# 测试跨组件依赖注入
print("1. 测试基础服务 (base组件):")
db_service = root_container.get(DatabaseService, component="base")
cache_service = root_container.get(CacheService, component="base")
print(f"数据库服务: {db_service.connect()}")
print(f"缓存服务: {cache_service.get_cache_info()}")

print("\n2. 测试仓库层 (repo组件，依赖base组件):")
user_repo = root_container.get(UserRepository, component="repo")
user_data = user_repo.get_user(123)
print(f"用户数据: {user_data}")

print("\n3. 测试服务层 (service组件，依赖repo组件):")
user_service = root_container.get(UserService, component="service")
user_profile = user_service.get_user_profile(456)
print(f"用户资料: {user_profile}")

print("\n4. 测试容器嵌套访问:")
# 创建子容器路径
sub_containers = AstrbotContainers()
_ = sub_containers << ("app/api/base.p", BaseServicesProvider())
_ = sub_containers << ("app/api/repositories.p", RepositoryProvider())
_ = sub_containers << ("app/api/services.p", ServiceProvider())

# 获取父容器
parent_container = sub_containers.getContainer("app")

# 获取子容器
child_container = sub_containers.getContainer("app/api")

# 子容器可以访问父容器的依赖
parent_db = parent_container.get(DatabaseService, component="base")
child_service = child_container.get(UserService, component="service")

print(f"父容器数据库: {parent_db.connect()}")
print(f"子容器服务: {child_service.get_user_profile(789)}")

print("\n✓ 嵌套依赖和容器测试完成!")

=== 嵌套依赖和容器测试 ===
DEBUG get_all_providers: self=2248498860816, self.providers={'message': None, 'context': <__main__.ContextProvider object at 0x0000020B85144EC0>, 'user_db': <__main__.UserProvider object at 0x0000020B854AAF90>, 'comment_db': <__main__.CommentProvider object at 0x0000020B854AB0E0>, 'main': <__main__.MainProvider object at 0x0000020B854AB230>, 'base_services': <__main__.BaseServicesProvider object at 0x0000020B854AB620>, 'repositories': <__main__.RepositoryProvider object at 0x0000020B854AB4D0>, 'services': <__main__.ServiceProvider object at 0x0000020B854AB770>}, all_providers={'message': None, 'context': <__main__.ContextProvider object at 0x0000020B85144EC0>, 'user_db': <__main__.UserProvider object at 0x0000020B854AAF90>, 'comment_db': <__main__.CommentProvider object at 0x0000020B854AB0E0>, 'main': <__main__.MainProvider object at 0x0000020B854AB230>, 'base_services': <__main__.BaseServicesProvider object at 0x0000020B854AB620>, 'repositories': <__main__.Repository

ProviderNotSetError: Provider not set: app/api